In [ ]:
import numpy as np
import pandas as pd
import re,math
import concurrent.futures
import multiprocessing
import threading
import logging
import time

# 设置日志级别
logging.basicConfig(level=logging.DEBUG)

# 设置函数运行计时用的修饰器
def timeit(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds')
        return result
    return wrapper

# 高相似多少
coum=200
# 加载保存的数组
lin = np.load("lin.npy")
# lin = np.linspace(0,3,401)
# lin = np.linspace(0.01,1,100)


data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'583drugs39cell Zscore行标准化 synergy.xlsx')
drug_names = list(data1.index)

# 合并drugA和drugB为一列作为索引
# 顺序
temp1 = [] # 分子指纹合并值 【【】，【】，【】，】
index_1 = [] # 对应的名字列表 【【】，【】，【】，】
# 逆序
temp2 = []
index_2 = []
for index,row in data2.iterrows():
    ct = list((np.array(data1.loc[row["DrugA"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugB"],:]))))
    temp1.append(ct)
    index_1.append((row["DrugA"],row["DrugB"]))
    ct = list((np.array(data1.loc[row["DrugB"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugA"],:]))))
    temp2.append(ct)
    index_2.append((row["DrugB"],row["DrugA"]))

/root/miniconda3/envs/jupyter-env/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [ ]:
# 两组数据的tanimoto计算
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]
        b=q[i]
        if (a==1)|(b==1): # 并
            tep1=tep1+1
        if (a==1)&(b==1): # 交
            tep2=tep2+1
    c=round((tep2 / tep1),4)      #取值4位数
    return c

def compute_tanimoto(i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx):
    tmp=[]
    a = np.array(i) #取出一分子指纹 转换列表为向量
    b = np.array(j)
    for k,m,k_n,m_n in zip(temp1,temp2,index_1,index_2): # 遍历每一个药物对的分子指纹
        c = np.array(k)
        d = np.array(m)
        tp1 = tanimoto(a,c)
        tp2 = tanimoto(a,d)
        tp3 = tanimoto(b,c)
        tp4 = tanimoto(b,d)
        tps = [tp1, tp2, tp3, tp4]
        index_list = [(i_n,k_n,tp1),(i_n,m_n,tp2),(j_n,k_n,tp3),(j_n,m_n,tp4)]
        index_location = tps.index(max(tps))
        tmp.append(max(tps))
        Tanimoto_index_dist[f"{i_n},{k_n}"] = index_list[index_location]
    return idx, tmp

temp = [None] * len(temp1)
Tanimoto_index_dist = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 提交任务时使用 enumerate ，并将索引值作为参数传递给任务函数
    futures = [executor.submit(compute_tanimoto, i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx)
               for idx, (i,j,i_n,j_n) in enumerate(zip(temp1,temp2,index_1,index_2))]
    for future in concurrent.futures.as_completed(futures):
        idx, result = future.result()
        temp[idx] = result
Tanimoto = pd.DataFrame(temp, columns=index_1, index=index_1)

In [ ]:
# 获取columns数组 - 处理后的
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1]) # 去除首尾括号的药物对名字
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp)) # 药物对协同得分
columns = data2.columns # 修改后的药物对名字

In [ ]:
# 删去对应行后的字典
dict_lines = {} # {键1：值1} 谷本系数字典
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct]) # 每一列谷本系数 [:,1] 第一列的每一行 实际就是第一列
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y # 谷本系数药物对的名字做键1，y是值1
    ct += 1

In [ ]:
# 分离x y数据构造字典
dict_columns_split = {} # 高相似字典
# 计算x数据
x = data2.drop(columns=columns)  # 其他所有的列
for i in columns:
    # b 要预测协同得分那一列
    y = data2.loc[:, i]
    # A 权重
    x_i = x.copy()
    x_i['y'] = y

    # 高相似的    取高相似的药物对名字max1和对应的值max5
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_lines[i], index = tp)
    df.sort_values(by=0, inplace=True, ascending=True)
    max_l = df.index[-coum:]
    max_5x = pd.DataFrame([data2.loc[:, j] for j in max_l]).T

    # 存储 对应数据
    count = 0 # 第几行 int
    for j in list(y.index): # 35个索引 0~34 数字（字符串-str） 【0，1，2，3.。。。】
        dict_columns_split[f'{i},{j}']={} # 空字典 键-空字典
        dict_columns_split[f'{i},{j}']['y_line']=y[count]
        dict_columns_split[f'{i},{j}']['max_x_line']=max_5x.iloc[count,:].values
        count += 1

In [ ]:
dict_columns_split["'SUNITINIB', 'MK-4827',32"]['max_x_line']

array([ 9.0132660e-02, -9.8969820e-02, -2.2030113e+00, -2.6533191e+00,
       -1.3656538e+00, -2.5928310e+00, -4.1963280e-02, -6.7495495e-01,
       -1.0962662e+00, -3.2805324e-01, -2.4593820e-01, -5.4777384e-01,
       -3.4914732e-01, -2.6289720e+00, -3.5352638e-01, -6.3626295e-01,
        2.5722393e-01,  2.7945230e-01,  1.1201497e+00, -1.2683841e+00,
       -1.5246437e+00, -8.9764000e-04, -1.2019012e+00, -1.7924271e+00,
        1.0922955e+00, -9.8660110e-01,  5.5708370e-01,  4.5372880e-01,
       -3.1579345e-01, -9.3183120e-01,  2.1274490e-01, -4.9613390e-01,
       -5.2834000e-01, -7.5484276e-01, -1.7530004e+00,  7.2603956e-02,
       -3.8278688e-02, -2.2323730e+00, -1.5543456e+00, -2.2530637e+00,
       -3.7979860e-01, -1.1786811e+00,  3.6453614e-01, -2.6384280e-01,
        2.0963123e-01,  1.5155060e-01, -6.2854420e-01,  4.4618350e-01,
       -6.8151623e-01,  8.6943164e-02,  1.2990282e+00,  1.8029979e-01,
       -9.0274096e-01,  2.7012500e-04, -1.7904884e+00, -4.8966548e-01,
      

In [ ]:
TTT = [] # 值列表
cct = 0
for i in Tanimoto.index: 
    y = list(Tanimoto.iloc[:,cct])
    y.pop(cct) 
    tp_columns = list(columns.copy())
    tp_columns.pop(cct)
    df = pd.DataFrame(y, index=tp_columns)
    df.sort_values(by=0 , inplace=True, ascending=True) 
    max_l = df.index[-coum:]  
    # 计算max_5y
    max_5y = []
    for j in max_l:
        max_5y.append(Tanimoto.iloc[list(columns).index(j), cct])
    # 将max_5y加入TTT
    TTT += max_5y
    # 修改cct变量的值
    cct += 1

In [ ]:
def alpha_pcc(t_coum, a, lin, linename, dict_columns_split, TTT, coum, data2):
    alpha = []
    result_pcc = []
    pcc_split = []
    for alp in lin:
        pre = []
        for elem in range(39):
            pp = [] 
            if TTT[(coum * a - t_coum):(coum * a)] == []:
                print("ERROR-TTT")
            for j in TTT[(coum * a - t_coum):(coum * a)]: 
                pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))
            pp = np.array(pp)  # 将 pp 转换为 numpy 数组
            pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
        pre = np.array(pre)  # 将 pre 转换为 numpy 数组
        pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
    result_pcc.append(np.max(pcc_split))
    alpha.append(lin[pcc_split.index(np.max(pcc_split))])
    pcc=result_pcc[0]
    del result_pcc, pcc_split, pre, pp
    return alpha, pcc

def alp_mse_rmse(alpha, t_coum, a, linename, dict_columns_split, TTT, coum):
    r1 = []  # 初始化 r1 为空列表
    alp = alpha[0]
    for elem in range(39):
        pp = []
        for j in TTT[(coum * a - t_coum):(coum * a)]: 
            try:
                pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  
            except:
                print(linename)
        r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) - dict_columns_split[f"{linename},{elem}"]['y_line'])
    r1 = np.array(r1)  # 将 r1 转换为 numpy 数组
    mse = np.mean(np.square(r1))
    rmse = math.sqrt(mse)
    del r1, pp,
    return alp, mse, rmse

In [ ]:
# 单进程 分函数
def calculate_parameters(t_coum, coum, TTT, data2, dict_columns_split, lin):
    a = 1
    Name_data = []
    for linename in columns:  
        # 求PCC
        alpha, pcc = alpha_pcc(t_coum, a, lin, linename, dict_columns_split, TTT, coum, data2)
        # 求MSE
        alp, mse, rmse = alp_mse_rmse(alpha, t_coum, a, linename, dict_columns_split, TTT, coum)
        Name_data.append([linename, t_coum, alp, mse, pcc, rmse])
        a = a+1
    pcc_mean = pd.DataFrame(Name_data, columns=['drug', 'coum', "alp", "mse", "pcc", "rmse"])["pcc"].mean()
    print(pcc_mean)
    return [pcc_mean, Name_data]

In [20]:
# 多进程
@timeit
def run_parallel(columns, lin):
    with multiprocessing.Pool() as pool:
        tp = []
        for t_coum in range(2, coum+1, 1):
            tp.append((t_coum, coum, TTT, data2, dict_columns_split, lin,))
        results = pool.starmap(calculate_parameters, tp)
    return results

PCC_mean = 0
res_pcc_list = []
results = run_parallel(columns,lin)
sorted_list = sorted(resultst, key=lambda x: x[0])
max_res = sorted_list[-1][1]
print("MAX PCC: ", sorted_list[-1][0])
# 将结果保存到 CSV 文件中
pd.DataFrame(max_res).to_csv('pcc_cpu_test_multi_unite_pcc.csv', header=None, index=False)

0.3142500958941148
0.4556292008643015
0.500061727057898
0.5343454711060138
0.37454724794995586
0.54187849472294
0.5148186827373478
0.5497992985275926
0.4273017168616017
0.5576210238782491
0.5699438036218804
0.4666124954923257
0.5738121099748807
0.3426566539121765
0.48155907210627025
0.5793165249645634
0.40690929594445285
0.5642842015378621
0.5942813534892257
0.5990591817841618
0.5835475146147665
0.589369344307911
0.608634121348226
0.5463952549869657
0.5395826917397204
0.5540654097463966
0.4441529970710218
0.5606133774831854
0.6108046357385177
0.5065595210794336
0.6046066147221084
0.5732278795836204
0.6058220499054262
0.6081655037374505
0.5772040036775803
0.5676095251196869
0.6112902818789873
0.5239314913458328
0.6104343588910714
0.6107374240617613
0.49130014239199965
0.5807978859570335
0.6116522588530466
0.6015460151864009
0.6031765917636763
0.6068605176313306
0.592293987343478
0.596496164455467
0.5857704226862201
0.6094928331524673
0.608762415357879
0.5998508961290129
0.60575379885261

In [25]:
pd.DataFrame(max_res)[4].mean()

0.6120428636469161

In [ ]:

# #print(TTT[70-10:70])

# # 加载保存的数组
# lin = np.load("lin.npy")
# PCC_mean = 0
# res_pcc_list = []
# for t_coum in range(2, coum+1, 1):    #邻居
#     PCC = 0
#     Name_data = []
#     a = 1
#     for linename in columns:
#         #求PCC
#         alpha = []
#         result_pcc = []
#         pcc_split = []
#         for alp in lin:
#             # r1 = []
#             # ct = 0
#             pre = []
#             for elem in range(39):
#                 pp = []
#                 for j in TTT[(coum * a - t_coum):(coum * a)]: 
#                     pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))#改函数
#                 pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
#             pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
#         result_pcc.append(np.max(pcc_split))
#         alpha.append(lin[pcc_split.index(np.max(pcc_split))])
#         pcc=result_pcc[0]

#         #求MSE
#         r1 = []
#         ct = 0
#         alp = alpha[0]
#         for elem in range(39):
#             pp = []
#             for j in TTT[(coum * a - t_coum):(coum * a)]:
#                 pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  # 改函数
#             r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) -
#                             dict_columns_split[f"{linename},{elem}"]['y_line'])
#         mse=np.mean(np.square(r1))
#         rmse=math.sqrt(mse)
#         Name_data.append([linename, t_coum, alp, mse, pcc, rmse])
#         a = a+1
#     pcc_mean = pd.DataFrame(Name_data, columns=['drug', 'coum', "alp", "mse", "pcc", "rmse"])["pcc"].mean()
#     if pcc_mean > PCC_mean:
#         PCC_mean = pcc_mean
#         res_pcc_list = Name_data
# pcc_mean = pd.DataFrame(res_pcc_list, columns=['drug', 'coum', "alp", "mse", "pcc", "rmse"]).to_csv('pcc_unite.csv', header=None, index=False)

In [ ]:
# # 多线程
# @timeit
# def run_parallel(columns, a):
#     threads = []
#     a = 1
#     for column in columns:
#         thread = threading.Thread(target=calculate_parameters, args=(column, a, coum, TTT, data2, dict_columns_split))
#         threads.append(thread)
#         thread.start()
#         a += 1
    
#     # 等待所有线程完成
#     for thread in threads:
#         thread.join()

# # 加载保存的数组
# lin = np.load("lin.npy")
# run_parallel(columns, a)    

# # 将结果保存到 CSV 文件中
# pd.DataFrame(results).to_csv('pcc_cpu_test_multi.csv', header=None, index=False)

In [ ]:
# def calculate_result(linename, Name_coum):
#     t_coum = Name_coum
#     # 求PCC
#     alpha = []
#     result_pcc = []
#     pcc_split = []
#     for alp in lin:
#         pre = []
#         for elem in range(39):
#             pp = [] 
#             for j in TTT[(coum * a - t_coum):(coum * a)]: 
#                 pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))
#             pp = np.array(pp)  # 将 pp 转换为 numpy 数组
#             pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
#         pre = np.array(pre)  # 将 pre 转换为 numpy 数组
#         pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
#     result_pcc.append(np.max(pcc_split))
#     alpha.append(lin[pcc_split.index(np.max(pcc_split))])
#     pcc=result_pcc[0]
#     # 求MSE
#     r1 = []  # 初始化 r1 为空列表
#     alp = alpha[0]
#     for elem in range(39):
#         pp = []
#         for j in TTT[(coum * a - t_coum):(coum * a)]: 
#             pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  
#         r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) - dict_columns_split[f"{linename},{elem}"]['y_line'])
#     r1 = np.array(r1)  # 将 r1 转换为 numpy 数组
#     mse = np.mean(np.square(r1))
#     rmse = math.sqrt(mse)
#     return [linename, Name_coum, alp, mse, pcc, rmse]

# @timeit
# def run_parallel(columns, a):
#     with multiprocessing.Pool(processes=2) as pool:
#         results = []
#         for i, column in enumerate(columns):
#             result = pool.apply(calculate_parameters, (column, a + i, coum))
#             results.append(result)
#     return results